In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# ----------------------------------------
# 1. Generate Synthetic Hyperspectral Data
# ----------------------------------------
np.random.seed(42)
n_samples = 1000
n_bands = 100  # Hyperspectral bands

# 5 synthetic tree species
species = ['Mahogany', 'Teak', 'Pine', 'Ebony', 'Oak']
n_classes = len(species)

# Generate synthetic reflectance values
X = np.random.rand(n_samples, n_bands)

# Randomly assign species
y = np.random.choice(species, n_samples)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# ----------------------------------------
# 2. Train-Test Split
# ----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Reshape for 1D CNN input
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# ----------------------------------------
# 3. Define Deep Learning Model
# ----------------------------------------
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(n_bands, 1)),
    MaxPooling1D(2),
    Dropout(0.3),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(n_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ----------------------------------------
# 4. Train the Model
# ----------------------------------------
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

# ----------------------------------------
# 5. Evaluate the Model
# ----------------------------------------
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# ----------------------------------------
# 6. Plot Training History
# ----------------------------------------
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()
